In [ ]:
# Make Jupyter Notebook full screen 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
# Import statements:

import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

In [ ]:
# Local data paths:

GTFS_DATA_PATH = "mmt_gtfs/"
import os; os.listdir(GTFS_DATA_PATH) # Peak whats in the directory 

In [ ]:
def getPandasDFCSV(path, file, sep=','):  
    """ Keyword Arg sep: deliminator used in txt file (default = ',')"""
    pandasDF = pd.read_csv(path +  file, sep=sep)
    return pandasDF

In [ ]:
stop_times_df = getPandasDFCSV(GTFS_DATA_PATH, 'stop_times.txt')
#stop_times_df.info()
#stop_times_df.head(3)
trips_df = getPandasDFCSV(GTFS_DATA_PATH, 'trips.txt')
#trips_df.info()
#trips_df.head(3)
stops_df = getPandasDFCSV(GTFS_DATA_PATH, 'stops.txt')
#stops_df.info()
#stops_df.head(3)
trips_df = getPandasDFCSV(GTFS_DATA_PATH, 'trips.txt')
#trips_df.info()
#trips_df.head(3)

In [ ]:
# list of possible trips:

service_id_list = []

for i in trips_df['service_id']:
    if i not in service_id_list:
        service_id_list.append(i)
print(service_id_list)

In [ ]:
# list of possible routes:

route_short_name_list = []

for i in trips_df['route_short_name']:
    if i not in route_short_name_list:
        route_short_name_list.append(i)

route_short_name_list.sort()
print(route_short_name_list)

In [ ]:
trips_df_weekday_peak = trips_df[trips_df['service_id'] == '92_WKD']
trips_df_weekday_peak_R = trips_df[trips_df['service_id'] == '92_WKD:R']
trips_df_weekday_peak_s = trips_df[trips_df['service_id'] == '92_WKD:L#6']
trips_df_weekday_peak_s3 = trips_df[trips_df['service_id'] == '92_WKD:L=6']
trips_df_weekday_peak_s1 = trips_df[trips_df['service_id'] == '92_WKD:S']
trips_df_weekday_peak_s2 = trips_df[trips_df['service_id'] == '92_WKD:S#6']
trips_df_weekday_peak_s4 = trips_df[trips_df['service_id'] == '92_WKD:S=6']

dfs_wk = [trips_df_weekday_peak, trips_df_weekday_peak_R, trips_df_weekday_peak_s, 
          trips_df_weekday_peak_s1, trips_df_weekday_peak_s2, trips_df_weekday_peak_s4]

trips_df_weekday = pd.concat(dfs_wk)
#trips_df_weekday.head()

In [ ]:
routes_wk_list = []
for route in trips_df_weekday['route_short_name']:
    if route not in routes_wk_list:
        routes_wk_list.append(route)

routes_wk_list.sort()
print(routes_wk_list)

In [ ]:
print("weekend and holiday routes:")
for route in route_short_name_list:
    if route not in routes_wk_list:
        print(route)

In [ ]:
# make a dictionary of routes with trip ID's for weekday service:

tripIDList_forRoutes_weekday92 = dict()

for i in routes_wk_list:
    trip_list = []
    for j in trips_df_weekday[trips_df_weekday['route_short_name'] == i].iterrows():
        trip_list.append(j[1]['trip_id'])
    tripIDList_forRoutes_weekday92[i] = trip_list

#tripIDList_forRoutes_weekday92[23]

In [ ]:
for route in tripIDList_forRoutes_weekday92:
    
    for trip in tripIDList_forRoutes_weekday92[route]:
        trip_stop_time_df = stop_times_df[stop_times_df['trip_id'] == trip]

In [ ]:
def get_stop_distances(trip):
    """ getting the stop spacing for a given trip. Stop spacing is defined as the distance travelled by bus between two cosecutive stops. 
    Should mirror natural walking terrain better than distance between two globe points.
    
    return: numpy array - contains # of stops - 1 elements that are the distances between stops driven by the bus
    """
    stop_dist_array = []
    trip_stop_time_df = stop_times_df[stop_times_df['trip_id'] == trip]
    distance_travelled = np.array(trip_stop_time_df['shape_dist_traveled'])
    for i in range(len(distance_travelled)-1):
        stop_spacing = distance_travelled[i+1] - distance_travelled[i]
        stop_dist_array.append(stop_spacing)
        
    return stop_dist_array

print(get_stop_distances(9999999))

In [ ]:
all_trip_distances_forRoutes_weekday92 = dict()

for route in tripIDList_forRoutes_weekday92:
    #print(route)
    all_trip_distances = []
    for trip in tripIDList_forRoutes_weekday92[route]:
        #print(trip)
        trip_distances = get_stop_distances(trip)
        all_trip_distances.append(trip_distances)
    all_trip_distances_forRoutes_weekday92[route] = all_trip_distances
    
#all_trip_distances_forRoutes_weekday92

In [ ]:
all_trip_distances_forRoutes_weekday92 = dict()

for route in tripIDList_forRoutes_weekday92:
    #print(route)
    all_trip_distances = []
    for trip in tripIDList_forRoutes_weekday92[route]:
        #print(trip)
        trip_distances = get_stop_distances(trip)
        all_trip_distances += trip_distances
        all_trip_distances.sort()
    all_trip_distances_forRoutes_weekday92[route] = np.array(all_trip_distances)
    
#all_trip_distances_forRoutes_weekday92

In [ ]:
weekday_routes_92_allTrips = pd.DataFrame.from_dict(dict([ (k, pd.Series(v)) for k, v in all_trip_distances_forRoutes_weekday92.items()]))
#weekday_routes_92_allTrips

In [ ]:
ax = plt.subplots(figsize=(20, 8))[1]
yscale = 4

for i, col in enumerate(weekday_routes_92_allTrips.columns):
    s = weekday_routes_92_allTrips[col]
    x = i+1
    
    #y = s.quantile(0.5)
    #ax.plot(x, y, color='#ff9408', linewidth=3, markersize=5, marker = "_", alpha=.9, zorder=4)

    y1 = s.quantile(0.75)/yscale
    y2 = s.quantile(1)/yscale
    #ax.plot((x, x), (y1, y2), color='#87ae73', linewidth=10, markersize=1, linestyle = "-", alpha=.9)
    ax.axvline(x, ymin=y1, ymax=y2, color='#b6df00', linewidth=10, markersize=1, linestyle = "-", alpha=1, zorder=1)
    
    y1 = s.quantile(0.25)/yscale
    y2 = s.quantile(0.75)/yscale
    ax.axvline(x, ymin=y1, ymax=y2, color='#42928d', linewidth=10, markersize=1, linestyle = "-", alpha=1, zorder=2)
    #ax.plot((x, x), (y1, y2), color='#b00149', linewidth=10, markersize=1, linestyle = "-", alpha=.9)

    y1 = s.quantile(0)/yscale
    y2 = s.quantile(0.25)/yscale
    ax.axvline(x, ymin=y1, ymax=y2, color='#462d7c', linewidth=10, markersize=1, linestyle = "-", alpha=1, zorder=1)
    #ax.plot((x, x), (y1, y2), color='#34013f', linewidth=10, markersize=1, linestyle = "-", alpha=.9)

ax.axhline(y=0.2585, xmin=0, xmax=47, color="#be0119", alpha=.5, linestyle=":", lw=3, zorder=4)  

ax.set_xlim(0.5, len(weekday_routes_92_allTrips.columns)+0.5)
ax.set_ylim(0, yscale)
ax.set_xticklabels(list(weekday_routes_92_allTrips.columns), fontsize=14)
ax.set_xticks(range(1, len(weekday_routes_92_allTrips.columns)+1))
ax.set_yticklabels([0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0], fontsize=14)
    
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.xlabel("Weekday Bus Routes", fontsize=16)
plt.ylabel("Stop Distance (miles)", fontsize=16)

#plt.grid(axis='y', linestyle='--')

custom_lines = [Line2D([0], [0], color='#472777', lw=4), Line2D([0], [0], color='#1f918b', lw=4), Line2D([0], [0], color='#bedf23', lw=4), 
                Line2D([0], [0], color='#be0119', lw=4, linestyle=":")]
plt.legend(custom_lines, ['Bottom 25%', 'Middle 50%', 'Top 25%', 'Average'], fontsize=12)

plt.title("Stop Spacing Distribution by Route", fontsize=18)
plt.savefig("PaperFigures/Quartiles_allTrips_Weekdays", bbox_inches='tight')
plt.savefig("PaperFigures/Quartiles_allTrips_Weekdays", bbox_inches='tight', format='svg')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))
yscale = .8

for i, col in enumerate(weekday_routes_92_allTrips.columns):
    s = weekday_routes_92_allTrips[col]
    x = i+1

    y1 = s.quantile(0.75)/yscale
    y2 = s.quantile(1)/yscale
    #ax.plot((x, x), (y1, y2), color='#87ae73', linewidth=10, markersize=1, linestyle = "-", alpha=.9)
    ax.axvline(x, ymin=y1, ymax=y2, color='#b6df00', linewidth=10, markersize=1, linestyle = "-", alpha=1, zorder=2)
    
    y1 = s.quantile(0.25)/yscale
    y2 = s.quantile(0.75)/yscale
    ax.axvline(x, ymin=y1, ymax=y2, color='#42928d', linewidth=10, markersize=1, linestyle = "-", alpha=1, zorder=3)
    #ax.plot((x, x), (y1, y2), color='#b00149', linewidth=10, markersize=1, linestyle = "-", alpha=.9)

    y1 = s.quantile(0)/yscale
    y2 = s.quantile(0.25)/yscale
    ax.axvline(x, ymin=y1, ymax=y2, color='#462d7c', linewidth=10, markersize=1, linestyle = "-", alpha=1, zorder=2)
    #ax.plot((x, x), (y1, y2), color='#34013f', linewidth=10, markersize=1, linestyle = "-", alpha=.9)
    
ax.axhline(y=0.2585, xmin=0, xmax=47, color="#be0119", alpha=.7, linestyle=":", lw=3, zorder=4)   

ax.set_xlim(0.5, len(weekday_routes_92_allTrips.columns)+0.5)
ax.set_ylim(0, yscale)
ax.set_xticklabels(list(weekday_routes_92_allTrips.columns), fontsize=14)
ax.set_xticks(range(1, len(weekday_routes_92_allTrips.columns)+1))
    
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.xlabel("Weekday Bus Routes", fontsize=16)
plt.ylabel("Stop Distance (miles)", fontsize=16)

plt.yticks([.1, .2, .3, .4, .5, .6, .7, .8], fontsize=14)
plt.grid(axis='y', linestyle='--', zorder=0)

custom_lines = [Line2D([0], [0], color='#472777', lw=4), Line2D([0], [0], color='#1f918b', lw=4), Line2D([0], [0], color='#bedf23', lw=4), 
                Line2D([0], [0], color='#be0119', lw=4, linestyle=":")]
#plt.legend(custom_lines, ['Bottom 25%', 'Middle 50%', 'Top 25%', 'Average'])

#plt.title("Stop Spacing Distribution by Route", fontsize=18)
plt.savefig("PaperFigures/Quartiles_allTrips_Weekdays_zoom", bbox_inches='tight')
plt.savefig("PaperFigures/Quartiles_allTrips_Weekdays_zoom", bbox_inches='tight', format='svg')

plt.show()